In [ ]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Explainable AI

This notebook contains experiments on AI explainability using the following projects:

- 1. <a href='#intrepret_ml'>Intrepret ML</a>  
- 2. <a href='#intrepret_ml_community'>Intrepret ML Community</a> (has extra features)
- 3. <a href='#lime'>LIME</a>
- 4. <a href='#shap'>SHAP</a>

## Load and process data

In [ ]:
columns = [
    'status_checking_account',
    'duration_in_months',
    'credit_history',
    'purpose',
    'credit_amount',
    'savings_account_or_bonds',
    'present_employment_since',
    'installment_rate_in_percentage_of_disposable_income',
    'personal_status_and_sex',
    'other_debtors_guarantors',
    'present_residence_since',
    'property',
    'age_in_years',
    'other_installment_plans',
    'housing',
    'nr_existing_credits_at_this_bank',
    'job',
    'nr_people_liable_to_provide_maintenance_for',
    'telephone',
    'foreign_worker',
    'customer'
]

codes = {
    'A11': 'less than 0 DM',
    'A12': '0 - 200 DM',
    'A13': 'greater than 200 DM',
    'A14': 'no checking account',
    'A30': 'no credits taken/ all credits paid back duly',
    'A31': 'all credits at this bank paid back duly',
    'A32': 'existing credits paid back duly till now',
    'A33': 'delay in paying off in the past',
    'A34': 'critical account/ other credits existing (not at this bank)',
    'A40': 'car (new)',
    'A41': 'car (used)',
    'A42': 'furniture/equipment',
    'A43': 'radio/television',
    'A44': 'domestic appliances',
    'A45': 'repairs',
    'A46': 'education',
    'A47': 'vacation - does not exist?',
    'A48': 'retraining',
    'A49': 'business',
    'A410': 'others',
    'A61': 'less than 100 DM',
    'A62': '100 - 500 DM',
    'A63': '500 - 1000 DM',
    'A64': 'greater than 1000 DM',
    'A65': 'unknown/ no savings account',
    'A71': 'unemployed',
    'A72': 'less than 1 year',
    'A73': '1 - 4 years',
    'A74': '4 - 7 years',
    'A75': 'greater than 7 years',
    'A91': 'male divorced/separated',
    'A92': 'female divorced/separated/married',
    'A93': 'male single',
    'A94': 'male married/widowed',
    'A95': 'female single',
    'A101': 'none',
    'A102': 'co-applicant',
    'A103': 'guarantor',
    'A121': 'real estate',
    'A122': 'building society savings agreement/ life insurance',
    'A123': 'car or other',
    'A124': 'unknown / no property',
    'A141': 'bank',
    'A142': 'stores',
    'A143': 'none',
    'A151': 'rent',
    'A152': 'own',
    'A153': 'for free',
    'A171': 'unemployed/ unskilled - non-resident',
    'A172': 'unskilled - resident',
    'A173': 'skilled employee / official',
    'A174': 'management/ self-employed/highly qualified employee/ officer',
    'A191': 'none',
    'A192': 'yes, registered under the customers name',
    'A201': 'yes',
    'A202': 'no'
}

def process_data(df):
    
    df_ = df.copy()
    
    
    # LABEL ENCODE ORDINAL CATEGORICAL FEATURES

    # status_checking_account

    df_['has_checking_account'] = (df_['status_checking_account']!= 'no checking account').astype(int)

    status_checking_account_mapping = {'no checking account': 0,
                                       'less than 0 DM': 1,
                                       '0 - 200 DM': 2,
                                       'greater than 200 DM': 3}


    df_['status_checking_account'] = df_.status_checking_account.map(status_checking_account_mapping)


    # savings_account_or_bonds

    df_['known_savings_account_or_bonds'] = (df_['savings_account_or_bonds']!='unknown/ no savings account').astype(int)

    savings_account_or_bonds_mapping = {'unknown/ no savings account': 0,
                                       'less than 100 DM': 1,
                                       '100 - 500 DM': 2,
                                       '500 - 1000 DM': 3,
                                       'greater than 1000 DM': 4}

    df_['savings_account_or_bonds'] = df_.savings_account_or_bonds.map(savings_account_or_bonds_mapping)


    # present_employment_since

    df_['employed'] = (df_['present_employment_since']!='unemployed').astype(int)

    present_employment_since_mapping = {'unemployed': 0,
                                       'less than 1 year': 1,
                                       '1 - 4 years': 2,
                                       '4 - 7 years': 3,
                                       'greater than 7 years': 4}

    df_['present_employment_since'] = df_.present_employment_since.map(present_employment_since_mapping)

    
    
    # MERGE SIMILAR CATEGORIES

    df_['credit_history'] = df_['credit_history'].replace(
        {'all credits at this bank paid back duly':'no credits taken/ all credits paid back duly'})


    
    # FEATURE ENGINEERING CONSIDERING FEATURE INTERACTION

    df_['credit_amount/duration'] = df_.credit_amount/df_.duration_in_months
    df_['age/credit_amount'] = df_.age_in_years/df_.credit_amount

    df_['status_checking_account/age'] = df_.status_checking_account/df_.age_in_years
    df_['savings_account_or_bonds/age'] = df_.savings_account_or_bonds/df_.age_in_years

    df_['status_checking_account/credit_amount'] = df_.status_checking_account/df_.credit_amount
    df_['savings_account_or_bonds/credit_amount'] = df_.savings_account_or_bonds/df_.credit_amount


    
    # HANDLE RARE CATEGORIES IN CATEGORICAL FEATURES
    
    categoricals = df_.select_dtypes(include=['object']).columns.tolist()
        
    for cat in categoricals:
        frequencies = df_[cat].value_counts(normalize = True)
        mapping = df_[cat].map(frequencies).to_numpy()
        df_[cat] = df_[cat].mask(mapping < 0.02, 'other')

        
        
    # CHANGE TYPE OF CATEGORICAL FEATURES
    
    df_[categoricals] = df_.select_dtypes('object').astype('category')
    
    
    
    # PROCESS TARGET FEATURE (type of customer: bad: 1, good: 0)
    
    df_['customer'] = (df_.customer == 2).astype(int)
    
    return df_

In [ ]:
# data downloaded from https://archive.ics.uci.edu/ml/datasets/Statlog+%28German+Credit+Data%29
data = pd.read_csv('data/german.data', sep=' ', names=columns)
data = data.replace(codes)
data = process_data(data)

## EDA


In [ ]:
from interpret import show
from interpret.data import ClassHistogram

X = data.drop('customer', axis = 1)
y = data.customer

hist = ClassHistogram().explain_data(X, y, name = 'Train Data')
show(hist)

graphs = []
graphs.append(hist)

## Undersampling

In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

rus = RandomUnderSampler(random_state = 42)

X_, y_ = rus.fit_resample(X, y)

print(sorted(Counter(y_).items()))

## Train test split

In [ ]:
from sklearn.model_selection import train_test_split

seed = 42

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size = 0.2, random_state = seed)

# Modelling and Explanations

<a id='intrepret_ml'></a>

# *1. Using Interpret ML*

Interpret ML is an open-source package that incorporates state-of-the-art machine learning interpretability techniques under one roof.

Project: https://github.com/interpretml/interpret

Types of explanability algorithms available:
- Glassbox Models:
    - Inherently intelligible and explainable models (i.e Linear Models, Decision trees)
    - New model called Explainable Boosting Machine (EBM)
- Blackbox Models:
    - Generate explanations for any machine learning model
    - Uses interpretation techniques like LIME, SHAP, Partial Dependence, Morris Sensitivity

## 1.1. Glass Models

### 1.1.1. Explainable Boosting Classifier

parameters description: https://github.com/interpretml/interpret/blob/develop/python/interpret-core/interpret/glassbox/ebm/ebm.py

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from interpret.glassbox import ExplainableBoostingClassifier
from sklearn.model_selection import KFold


ebm = ExplainableBoostingClassifier(random_state = 42, 
                                    learning_rate = 0.01)

cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

cv_ebm = cross_val_score(ebm, X_train, y_train, cv = cv, scoring = 'accuracy')

print('Mean accuracy from CV: ', np.round(np.mean(cv_ebm), 2))

ebm.fit(X_train, y_train);

### 1.1.2. Logistic Regression

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from category_encoders.one_hot import OneHotEncoder
from interpret.glassbox import LogisticRegression


# Get features names after encoding
ohe = OneHotEncoder(use_cat_names = True)
ohe.fit(X_train)
feature_names = ohe.feature_names


lr = LogisticRegression(random_state = 42, 
                        solver = 'liblinear', 
                        C = 0.0133, 
                        penalty = 'l2',
                        feature_names = feature_names)


pipe_lr = Pipeline([
    ('ohe', OneHotEncoder(use_cat_names = True)),
    ('scaler', StandardScaler()),
    ('lr', lr)])


cv = KFold(n_splits = 5, shuffle = True, random_state = 42)


cv_lr = cross_val_score(pipe_lr, X_train, y_train, cv = cv, scoring = 'accuracy')

print('Mean accuracy from CV: ', np.round(np.mean(cv_lr), 2))

pipe_lr.fit(X_train, y_train);

### 1.1.3. Performance

In [ ]:
from sklearn.metrics import accuracy_score
from interpret.perf import ROC

acc_ebm = accuracy_score(y_test, ebm.predict(X_test))
acc_lr = accuracy_score(y_test, pipe_lr.predict(X_test))

print('Test accuracy: ')
print('EBM: ', np.round(acc_ebm, 4))
print('LR: ', np.round(acc_lr, 4))

ebm_perf = ROC(ebm.predict_proba).explain_perf(X_test, y_test, name = 'Explainable Boosting Classifier')

lr_perf = ROC(pipe_lr['lr'].predict_proba).explain_perf(X = pipe_lr[:2].transform(X_test),
                                                        y = y_test, 
                                                        name = 'Logistic Regression')

graphs += [ebm_perf, lr_perf]

### 1.1.4. Global explanations

In [ ]:
ebm_global = ebm.explain_global(name='Explainable Boosting Classifier')
lr_global = pipe_lr['lr'].explain_global(name='Logistic Regression')


graphs += [ebm_global, lr_global]

### 1.1.5. Local explanations

In [ ]:
ebm_local = ebm.explain_local(X_test[:10], y_test[:10], name = 'Explainable Boosting Classifier')

lr_local = pipe_lr['lr'].explain_local(X = pipe_lr[:2].transform(X_test[:10]),
                                       y = y_test[:10], 
                                       name = 'Logistic Regression')

graphs += [ebm_local, lr_local]

## 1.2. Black-box Models

### 1.2.1. SVM

In [ ]:
from sklearn.svm import SVC


svm = SVC(random_state = 42, C = 0.001, gamma = 0.001, kernel = 'linear', probability = True)

pipe_svm = Pipeline([
    ('ohe', OneHotEncoder(use_cat_names = True)),
    ('scaler', StandardScaler()),
    ('svm', svm)])

cv = KFold(n_splits = 5, shuffle = True, random_state = 42)

cv_svm = cross_val_score(pipe_svm, X_train, y_train, cv = cv, scoring = 'accuracy')

print('Mean accuracy from CV: ', np.round(np.mean(cv_svm), 2))

pipe_svm.fit(X_train, y_train);

### 1.2.2. RandomForestClassifier 

In [ ]:
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(random_state = 42,
                            bootstrap = False,
                            max_depth = 10,
                            max_features = 'auto',
                            min_samples_leaf = 4,
                            min_samples_split = 2,
                            n_estimators = 130)


pipe_rf = Pipeline([
    ('ohe', OneHotEncoder(use_cat_names = True)),
    ('rf', rf)
])

cv = KFold(n_splits = 5, shuffle = True, random_state = 42)
        
cv_rf = cross_val_score(pipe_rf, X_train, y_train, cv = cv, scoring = 'accuracy')

print('Mean accuracy from CV: ', np.round(np.mean(cv_rf), 2))

pipe_rf.fit(X_train, y_train);

acc_rf = accuracy_score(y_test, pipe_rf.predict(X_test))

### 1.2.4. Performance

In [ ]:
from interpret.perf import ROC

acc_svm = accuracy_score(y_test, pipe_svm.predict(X_test))
acc_rf = accuracy_score(y_test, pipe_rf.predict(X_test))

print('Test accuracy: ')
print('SVM: ', np.round(acc_svm, 4))
print('RF: ', np.round(acc_rf, 4))


svm_perf = ROC(pipe_svm['svm'].predict_proba).explain_perf(X = pipe_svm[:2].transform(X_test),
                                                           y = y_test,
                                                           name = 'SVM')

rf_perf = ROC(pipe_rf['rf'].predict_proba).explain_perf(X = pipe_rf[:1].transform(X_test),
                                                        y = y_test,
                                                        name = 'Random Forest Classifier')

graphs += [svm_perf, rf_perf]

### 1.2.5. Global explanations

In [ ]:
from interpret.blackbox import MorrisSensitivity
from interpret.blackbox import PartialDependence

def blackbox_global_explanations(predict_fn, data, feature_names, name):
    
    # Morris Sensitivity
    
    sensitivity = MorrisSensitivity(predict_fn, data, feature_names = feature_names)
    sensitivity_global = sensitivity.explain_global(name = name + " - Morris Sensitivity")
    
    # Partial Dependence
    
    pdp = PartialDependence(predict_fn, data, feature_names = feature_names)
    pdp_global = pdp.explain_global(name = name + " - Partial Dependence")
    
                                    
    return sensitivity_global, pdp_global

In [ ]:
svm_sensitivity_global, svm_pdp_global = blackbox_global_explanations(predict_fn = pipe_svm['svm'].predict_proba,
                                                                      feature_names = feature_names,
                                                                      data = pipe_svm[:2].transform(X_train), 
                                                                      name = "SVM")


rf_sensitivity_global, rf_pdp_global = blackbox_global_explanations(predict_fn = pipe_rf['rf'].predict_proba,
                                                                    feature_names = feature_names,
                                                                    data = pipe_rf[:1].transform(X_train),
                                                                    name = "RF")


graphs += [svm_sensitivity_global, svm_pdp_global, rf_sensitivity_global, rf_pdp_global]

### 1.2.6. Local explanations

In [ ]:
from interpret.blackbox import LimeTabular
from interpret.blackbox import ShapKernel

def blackbox_local_explanations(predict_fn, X_train, X_explain, y_explain, feature_names, name):
    
    
    # Lime
    
    lime = LimeTabular(predict_fn, data = X_train, random_state = seed, feature_names = feature_names)
    lime_local = lime.explain_local(X_explain, y_explain, name = name + ' - Lime')


    # Shap
    
    X_train_ = np.median(X_train, axis=0).reshape(1, -1)

    shap = ShapKernel(predict_fn, data = X_train_, feature_names = feature_names)
    shap_local = shap.explain_local(X_explain, y_explain, name = name + ' - Shap')

    
    return lime_local, shap_local

In [ ]:
svm_lime_local, svm_shap_local = blackbox_local_explanations(predict_fn = pipe_svm['svm'].predict_proba, 
                                                             X_train = pipe_svm[:2].transform(X_train), 
                                                             X_explain = pipe_svm[:2].transform(X_test[:5]),
                                                             y_explain = y_test[:5], 
                                                             feature_names = feature_names, 
                                                             name = 'SVM')

                                                             
rf_lime_local, rf_shap_local = blackbox_local_explanations(predict_fn = pipe_rf['rf'].predict_proba,
                                                           X_train = pipe_rf[:1].transform(X_train), 
                                                           X_explain = pipe_rf[:1].transform(X_test[:5]),
                                                           y_explain = y_test[:5],
                                                           feature_names = feature_names, 
                                                           name = 'Random Forest')


graphs += [svm_lime_local, svm_shap_local, rf_lime_local, rf_shap_local]

## 1.3. Summary

In [ ]:
models = ['Explainable Boosting Classifier', 'Logistic Regression', 'SVM', 'Random Forest Classifier']
acc_train = np.round([np.mean(cv_ebm), np.mean(cv_lr), np.mean(cv_svm), np.mean(cv_rf)], 4)
acc_test = np.round(np.array([acc_ebm, acc_lr, acc_svm, acc_rf]), 4)

summary = pd.DataFrame({'Models': models, 'Train Accuracy': acc_train, 'Test Accuracy': acc_test})
summary = summary.sort_values(by = 'Test Accuracy', ascending = False)
summary

## 1.4. Interpret ML Dashboard

In [ ]:
show(graphs, share_tables=True)

<a id='intrepret_ml_community'></a>

# *2. Using Interpret ML Community*

Interpret-Community is an experimental repository that extends Interpret with additional capabilities. 

Project: https://github.com/interpretml/interpret-community

### Raw features transformation

You can pass your feature transformation pipeline to the explainer to receive explanations in terms of the raw features before the transformation (rather than engineered features). If you skip this, the explainer provides explanations in terms of engineered features.

The format of supported transformations is same as the one described in sklearn-pandas. In general, any transformations are supported as long as they operate on a single column and are therefore clearly one to many.

We can explain raw features by either using a sklearn.compose.ColumnTransformer or a list of fitted transformer tuples.

## 2.1. Feature transformations and Model

In [ ]:
from interpret.ext.blackbox import KernelExplainer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder


# Create transformers for numeric and categorical features

numeric_features = X_train.select_dtypes(np.number).columns.tolist()
categorical_features = X_train.select_dtypes('category').columns.tolist()

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])


# Create pipeline with the transformer and the model

svm = SVC(random_state = 42, probability = True)

clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', svm)])


model = clf.fit(X_train, y_train)

print('accuracy: ', np.round(accuracy_score(y_test, model.predict(X_test)), 4))

## 2.2. Explanations

In [ ]:
tabular_explainer = KernelExplainer(clf.steps[-1][1],
                                     initialization_examples = X_train,
                                     features = X_train.columns.tolist(),
                                     classes = ['good customer', 'bad customer'],
                                     transformations = preprocessor)

### 2.2.1. Global explanations

In [ ]:
# Passing in test dataset for evaluation examples - note it must be a representative sample of the original data
# X_train can be passed as well, but with more examples explanations will take longer although they may be more accurate

global_explanation = tabular_explainer.explain_global(X_test)

global_explanation.visualize()

### 2.2.2. Local explanations

In [ ]:
def plot_local_explanations(sorted_local_importance_names, sorted_local_importance_values):
    
    x = np.array(sorted_local_importance_values[1][0])

    sorting = np.argsort(-np.abs(x))[:15]
    
    y = np.array(sorted_local_importance_names[1][0])
    
    hue = (x > 0)*1

    plt.figure(figsize = (8,8))
    sns.barplot(x = x[sorting], y = y[sorting], hue = hue[sorting]);
    
    
sample_idx = 7
sample = X_test.reset_index(drop=True).iloc[[sample_idx]]

local_explanation = tabular_explainer.explain_local(sample)

print('True class: ', y_test.iloc[sample_idx])
print('Predicted proba: ', np.round(model.predict_proba(sample)[0][1], 2))

sorted_local_importance_names = local_explanation.get_ranked_local_names()
sorted_local_importance_values = local_explanation.get_ranked_local_values()

plot_local_explanations(sorted_local_importance_names, sorted_local_importance_values)

### 2.2.3. Explanation Dashboard

In [ ]:
from interpret_community.widget import ExplanationDashboard

ExplanationDashboard(global_explanation, model, datasetX = X_test, true_y = y_test);

<a id='lime'></a>

# *3. Using LIME*

Project: https://github.com/marcotcr/lime

## 3.1 Feature transformations and model


One hot encode the categorical features for the classifier, but not for the explainer. The explainer should have the categorical features label encoded instead, because it must make sure that a categorical feature only has one possible value when perturbing the data. For example, if after one hot encoding the data [0,1] is female and [1,0] is male, a perturbation [1,1] caused by LIME algorithm would not make sense.

In [ ]:
import lime
import lime.lime_tabular
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


X_enc = X_.copy()


# LABEL ENCODE CATEGORICAL FEATURES
# build dictionary to save the correspondence between labels and the original string classes

categorical_features = X_enc.select_dtypes('category').columns.tolist()

categorical_names = {}

for feature in categorical_features:
    le = LabelEncoder()
    le.fit(X_enc[feature])
    X_enc[feature] = le.transform(X_enc[feature])
    categorical_names[feature] = le.classes_
    

# Get index of categorical features   
feature_names = X_enc.columns.tolist()
categorical_features_idx = pd.Series(feature_names).isin(categorical_features)
categorical_features_idx = np.arange(len(feature_names))[categorical_features_idx].tolist()


# SPLIT DATA INTO TRAIN AND TEST SETS

X_train_enc, X_test_enc, y_train, y_test = train_test_split(X_enc, y_, test_size = 0.2, random_state = 42)


# ONE HOT ENCODE THE DATA FOR THE CLASSIFIER

ohe = OneHotEncoder(handle_unknown = 'ignore')

X_train_ohe = ohe.fit_transform(X_train_enc)
X_test_ohe = ohe.transform(X_test_enc)

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train_ohe, y_train);


print('accuracy: ', np.round(accuracy_score(y_test, rf.predict(X_test_ohe)), 4))

## 3.2 Explanation

LimeTabularExplainer needs a training set because LIME computes statistics on each feature. If the feature is numerical, LIME computes the mean and std, and discretizes it into quartiles. If the feature is categorical, LIME computes the frequency of each value.

These statistics are computed for two things:

1. To scale the data, so that we can meaningfully compute distances when the attributes are not on the same scale
2. To sample perturbed instances - which we do by sampling from a Normal(0,1), multiplying by the std and adding back the mean.


Find documentation on parameters info here: https://lime-ml.readthedocs.io/en/latest/lime.html

In [ ]:
# Train data should be label encoded not one hot encoded

np.random.seed(1)

explainer = lime.lime_tabular.LimeTabularExplainer(X_train_enc.to_numpy(),
                                                   feature_names = feature_names,
                                                   class_names = ['good customer', 'bad customer'],
                                                   categorical_features = categorical_features_idx, 
                                                   categorical_names = categorical_names, 
                                                   kernel_width = 4)


In [ ]:
np.random.seed(1)

sample_idx = 5
sample = X_test_enc.reset_index(drop = True).to_numpy()[sample_idx]


# the predict function first transforms the data into the one-hot representation
predict_fn = lambda x: rf.predict_proba(ohe.transform(x)).astype(float)


exp = explainer.explain_instance(sample, predict_fn = predict_fn)
exp.show_in_notebook(show_all = True)


<a id='shap'></a>

# *4. Using SHAP*

Project: https://github.com/slundberg/shap  
Documentation: https://shap.readthedocs.io/en/latest/#

In [ ]:
import shap
import lightgbm as lgb

# Initialize your Jupyter notebook with initjs(), otherwise you will get an error message when plotting the graphs
shap.initjs()

## 4.1. Model

In [ ]:
d_train = lgb.Dataset(X_train, label = y_train)
d_test = lgb.Dataset(X_test, label = y_test)


params = {
    "num_iterations": 10000,
    "min_data_in_leaf": 50,
    "max_bin": 50,
    "learning_rate": 0.0001,
    "boosting_type": "gbdt",
    "objective": "binary",
    "metric": "binary_logloss",
    "num_leaves": 11,
    "max_depth": 10,
    "verbose": -1,
    "min_data": 100,
    "boost_from_average": True,
    "seed": 42,
    "feature_fraction": 0.4
}

model = lgb.train(params, d_train, valid_sets = [d_test], early_stopping_rounds = 50, verbose_eval = 1000)

print()
print('train accuracy: ', np.round(accuracy_score(y_train, model.predict(X_train).round(0)), 4))
print('test accuracy: ', np.round(accuracy_score(y_test, model.predict(X_test).round(0)), 4))

## 4.2. Explanations

### Explain entire dataset

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

### Visualize a single prediction

In [ ]:
sample_idx = 1
sample = X_test.reset_index(drop = True).iloc[[sample_idx]]

print('True class: ', y_test.reset_index(drop = True).iloc[sample_idx])
print('Predicted proba: ', np.round(model.predict(sample)[0], 2))

shap.force_plot(explainer.expected_value[1], shap_values[1][sample_idx,:], sample)


### Visualize many predictions

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values[1][:1000,:], X_train.iloc[:1000,:])

### SHAP summary plot

Density scatter plot of SHAP values for each feature to identify how much impact each feature has on the model output. Features are sorted by the sum of the SHAP value magnitudes across all samples.

In [ ]:
shap.summary_plot(shap_values, X_train)

### SHAP Dependence Plots

SHAP dependence plots show the effect of a single feature across the whole dataset. They plot a feature’s value vs. the SHAP value of that feature across many samples. SHAP dependence plots are similar to partial dependence plots, but account for the interaction effects present in the features, and are only defined in regions of the input space supported by data. The vertical dispersion of SHAP values at a single feature value is driven by interaction effects with other features, and another feature is chosen for coloring to highlight possible interactions. 

In [ ]:
numerical_features = X_test.select_dtypes(np.number).columns.tolist()
numerical_features_idx = X_test.columns.isin(numerical_features)

for name in numerical_features:
    shap.dependence_plot(name, shap_values[1][:,numerical_features_idx], X_test[numerical_features])